In [1]:
!pip freeze > requirements.txt

In [1]:
import pandas as pd
import numpy as np
import plotly
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.figure_factory as ff
import plotly.express as px

In [2]:
df = pd.read_csv('source/main_data.csv', sep=',', encoding = "ISO-8859-1", engine='python')

In [3]:
df.dropna(axis='index', how='any', inplace=True)

In [4]:
#df.assign(MyCountry = df['Country'] + 'MY')
df = df.astype({'CustomerID':'int32',
                'Quantity':'int32',
                'InvoiceDate':'datetime64[ns]'})

In [5]:
#df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Date'] = pd.to_datetime(df['InvoiceDate'].dt.strftime('%Y-%m-%d'))

# Получим дату первой покупки в отдельный столбец
df['FirstDate'] = df.groupby('CustomerID')['Date'].transform('min')
df['MonthDate'] = pd.to_datetime(df['FirstDate'].dt.strftime('%Y-%m'))

In [7]:
# Получим количесво календарных месяцев между первой датой покупки и текущей
def diff_month(row, date, first_date):
    d1 = row[date]
    d2 = row[first_date]
    r = abs((d1.year - d2.year) * 12 + d1.month - d2.month)
    return r

In [8]:
# Разность от первой даты
df['cohort_number'] = df.apply(diff_month, date='Date', first_date='FirstDate', axis=1)

# Количество когорт по датам
df['cohort_count'] = df.groupby(['MonthDate'])['cohort_number'].transform('nunique')

In [10]:
cohort_df = pd.pivot_table(df[df['cohort_count']>0], 
                           index='MonthDate', 
                           columns='cohort_number', 
                           aggfunc = {'CustomerID':'nunique'})

In [11]:
template = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]

In [12]:
# Количество
cohort_df

CustomerID                                                   \
cohort_number         0      1      2      3      4      5      6      7    
MonthDate                                                                   
2010-12-01         948.0  362.0  317.0  367.0  341.0  376.0  360.0  336.0   
2011-01-01         421.0  101.0  119.0  102.0  138.0  126.0  110.0  108.0   
2011-02-01         380.0   94.0   73.0  106.0  102.0   94.0   97.0  107.0   
2011-03-01         440.0   84.0  112.0   96.0  102.0   78.0  116.0  105.0   
2011-04-01         299.0   68.0   66.0   63.0   62.0   71.0   69.0   78.0   
2011-05-01         279.0   66.0   48.0   48.0   60.0   68.0   74.0   29.0   
2011-06-01         235.0   49.0   44.0   64.0   58.0   79.0   24.0    NaN   
2011-07-01         191.0   40.0   39.0   44.0   52.0   22.0    NaN    NaN   
2011-08-01         167.0   42.0   42.0   42.0   23.0    NaN    NaN    NaN   
2011-09-01         298.0   89.0   97.0   36.0    NaN    NaN    NaN    NaN   
2011-10-01         352.0   93.0   46.0    NaN    NaN    NaN    NaN    NaN   
2011-11-01         321.0   43.0    NaN    NaN    NaN    NaN    NaN    NaN   
2011-12-01          41.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                                                  
cohort_number     8      9      10     11     12  
MonthDate                                         
2010-12-01     336.0  374.0  354.0  474.0  260.0  
2011-01-01     131.0  146.0  155.0   63.0    NaN  
2011-02-01      98.0  119.0   35.0    NaN    NaN  
2011-03-01     127.0   39.0    NaN    NaN    NaN  
2011-04-01      25.0    NaN    NaN    NaN    NaN  
2011-05-01       NaN    NaN    NaN    NaN    NaN  
2011-06-01       NaN    NaN    NaN    NaN    NaN  
2011-07-01       NaN    NaN    NaN    NaN    NaN  
2011-08-01       NaN    NaN    NaN    NaN    NaN  
2011-09-01       NaN    NaN    NaN    NaN    NaN  
2011-10-01       NaN    NaN    NaN    NaN    NaN  
2011-11-01       NaN    NaN    NaN    NaN    NaN  
2011-12-01       NaN    NaN    NaN    NaN    NaN

In [13]:
# Количество
cohort_df['CustomerID'].T.iplot()

In [14]:
# Количество
t = template[0]
px.imshow(cohort_df['CustomerID'], text_auto=True, template=t, title=t).show()

In [15]:
def retention(row, column):
    res = round((row[column]/row[column][0])*100,2)
    return res
ret_df = cohort_df.apply(retention, column='CustomerID', axis=1)

In [16]:
# Удержание
ret_df

cohort_number,0,1,2,3,4,5,6,7,8,9,10,11,12
MonthDate,,,,,,,,,,,,,
2010-12-01,100.0,38.19,33.44,38.71,35.97,39.66,37.97,35.44,35.44,39.45,37.34,50.00,27.43
2011-01-01,100.0,23.99,28.27,24.23,32.78,29.93,26.13,25.65,31.12,34.68,36.82,14.96,NaN
2011-02-01,100.0,24.74,19.21,27.89,26.84,24.74,25.53,28.16,25.79,31.32,9.21,NaN,NaN
2011-03-01,100.0,19.09,25.45,21.82,23.18,17.73,26.36,23.86,28.86,8.86,NaN,NaN,NaN
2011-04-01,100.0,22.74,22.07,21.07,20.74,23.75,23.08,26.09,8.36,NaN,NaN,NaN,NaN
2011-05-01,100.0,23.66,17.20,17.20,21.51,24.37,26.52,10.39,NaN,NaN,NaN,NaN,NaN
2011-06-01,100.0,20.85,18.72,27.23,24.68,33.62,10.21,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,100.0,20.94,20.42,23.04,27.23,11.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,100.0,25.15,25.15,25.15,13.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Удержание
px.imshow(ret_df, text_auto=True).show()